In [ ]:
import torch
from torch.nn import Module, Linear, Dropout, ModuleList

from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.datasets import TUDataset

dataset_root = "./tmp"
dataset_name = "MUTAG"

# Loading MUTAG dataset
dataset = TUDataset(root=dataset_root, name=dataset_name)

# get data the first graph of the MUTAG
data = dataset[0]

In [ ]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T


dataset = TUDataset(root="./data", name="MUTAG")

# Print dataset properties for more understanding of the graph and his properties
print(f"Dataset: {dataset}")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of classes: {dataset.num_classes}")
print(f"Number of node features: {dataset.num_node_features}")

# Print properties of the first graph in the dataset
data = dataset[0]
print("\nFirst graph properties:")
print(f"Number of nodes: {data.num_nodes}")
print(f"Number of edges: {data.num_edges}")
print(f"Average node degree: {data.num_edges / data.num_nodes:.2f}")
print(f"Has isolated nodes: {data.has_isolated_nodes()}")
print(f"Has self-loops: {data.has_self_loops()}")
print(f"Is undirected: {data.is_undirected()}")

# we get and combine  properties for all graphs in the dataset
total_nodes = sum([graph.num_nodes for graph in dataset])
total_edges = sum([graph.num_edges for graph in dataset])
avg_nodes = total_nodes / len(dataset)
avg_edges = total_edges / len(dataset)

print("\nOverall dataset properties:")
print(f"Total number of nodes: {total_nodes}")
print(f"Total number of edges: {total_edges}")
print(f"Average number of nodes per graph: {avg_nodes:.2f}")
print(f"Average number of edges per graph: {avg_edges:.2f}")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool # Correct import statement
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.optim import Adam
import numpy as np
# ... rest of the code remains the same


dataset = TUDataset(root="./data", name="MUTAG")

# Verify dataset length
print(f"Total number of graphs in the dataset: {len(dataset)}")

# Print dataset properties get more undertanding
total_nodes = sum([graph.num_nodes for graph in dataset])
total_edges = sum([graph.num_edges for graph in dataset])
avg_nodes = total_nodes / len(dataset)
avg_edges = total_edges / len(dataset)
print(f"Number of classes: {dataset.num_classes}")
print(f"Number of node features: {dataset.num_node_features}")
print(f"Total number of nodes: {total_nodes}")
print(f"Total number of edges: {total_edges}")
print(f"Average number of nodes per graph: {avg_nodes:.2f}")
print(f"Average number of edges per graph: {avg_edges:.2f}")

# Split the dataset into train, validation, and test sets
torch.manual_seed(12345)
dataset = dataset.shuffle()
num_train = int(0.8 * len(dataset))
num_val = int(0.1 * len(dataset))
num_test = len(dataset) - num_train - num_val

train_dataset = dataset[:num_train]
val_dataset = dataset[num_train:num_train + num_val]
test_dataset = dataset[num_train + num_val:]

# Verifications of  sizes of each split
print(f"Number of training graphs: {len(train_dataset)}")
print(f"Number of validation graphs: {len(val_dataset)}")
print(f"Number of test graphs: {len(test_dataset)}")

# DataLoaders for train, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ETSA(nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, num_gcn_layers=1, attention_type='sigmoid', dropout_rate=0.5):
        super(ETSA, self).__init__()
        self.embedding_layer = nn.Linear(num_features, hidden_dim)
        self.num_gcn_layers = num_gcn_layers
        self.gcn_layers = nn.ModuleList([GCNConv(hidden_dim, hidden_dim) for _ in range(self.num_gcn_layers)])

        if attention_type == 'sigmoid':
            self.stage_1_attention = nn.Linear(hidden_dim, 1)
            self.stage_2_attention = nn.Linear(hidden_dim, 1)
        elif attention_type == 'softmax':
            self.stage_1_attention = nn.Linear(hidden_dim, hidden_dim)
            self.stage_2_attention = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(p=dropout_rate)
        self.final_gcn_layer = GCNConv(hidden_dim, num_classes)

    def forward(self, x, edge_index, batch):
        x = self.embedding_layer(x)
        for i in range(self.num_gcn_layers):
            x = F.relu(self.gcn_layers[i](x, edge_index))
            x = self.dropout(x)

        stage_1_attention_weights = torch.sigmoid(self.stage_1_attention(x))
        stage_1_representation = torch.sum(x * stage_1_attention_weights, dim=0)

        stage_2_attention_weights = torch.sigmoid(self.stage_2_attention(x))
        stage_2_representation = torch.sum(x * stage_2_attention_weights, dim=0)

        combined_representation = stage_1_representation + stage_2_representation

        x = self.final_gcn_layer(x, edge_index)

        # Aggregate node embeddings to graph-level embeddings
        x = global_mean_pool(x, batch)

        return x, combined_representation

# Model, criterion, optimizer
model = ETSA(num_features=dataset.num_node_features, hidden_dim=64, num_classes=dataset.num_classes, num_gcn_layers=2, attention_type='sigmoid', dropout_rate=0.5).to(device)
criterion = torch.nn.CrossEntropyLoss()  # Using CrossEntropyLoss for classification tasks
optimizer = Adam(model.parameters(), lr=0.005)

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, precision, recall, f1

def fit():
    model.train()
    total_loss = 0
    y_true, y_pred = [], []
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out, _ = model(data.x, data.edge_index, data.batch)

        # Ensure the output and target dimensions match
        out = out.squeeze()
        target = data.y

        assert out.size(0) == target.size(0), f"Output size: {out.size()}, Target size: {target.size()}"

        loss = criterion(out, target)
        total_loss += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()

        y_true.append(target.detach().cpu())
        y_pred.append(out.max(1)[1].detach().cpu())

    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    accuracy, precision, recall, f1 = compute_metrics(y_true, y_pred)
    return total_loss / len(train_loader.dataset), accuracy

@torch.no_grad()
def evaluate(loader):
    model.eval()
    y_true, y_pred, y_scores = [], [], []
    for data in loader:
        data = data.to(device)
        out, _ = model(data.x, data.edge_index, data.batch)

        out = out.squeeze()
        target = data.y

        y_true.append(target.cpu())
        y_pred.append(out.max(1)[1].cpu())
        y_scores.append(F.softmax(out, dim=1)[:, 1].cpu().numpy())

    y_true = torch.cat(y_true, dim=0).detach().numpy()
    y_pred = torch.cat(y_pred, dim=0).detach().numpy()
    y_scores = np.concatenate(y_scores, axis=0)

    accuracy, precision, recall, f1 = compute_metrics(y_true, y_pred)
    auc = compute_auc(y_true, y_scores)
    return accuracy, precision, recall, f1, auc, y_scores

import matplotlib.pyplot as plt

# Lists to store metrics for plotting for the research paper , if more metrics are needed we will add them in this part.
train_losses = []
train_accuracies = []
val_accuracies = []
val_precisions = []
val_recalls = []
val_f1s = []
val_aucs = []

# Training loop
for epoch in range(201):
    loss, train_accuracy = fit()
    val_accuracy, val_precision, val_recall, val_f1, val_auc = evaluate(val_loader)[:5]  # Exclude y_scores from evaluation result

    # Store metrics
    train_losses.append(loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)
    val_precisions.append(val_precision)
    val_recalls.append(val_recall)
    val_f1s.append(val_f1)
    val_aucs.append(val_auc)

    if epoch % 50 == 0:
        print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {train_accuracy:.2f}% | Val Acc: {val_accuracy:.2f}% | Val Precision: {val_precision:.4f} | Val Recall: {val_recall:.4f} | Val F1: {val_f1:.4f} | Val AUC: {val_auc:.4f}')

# Final evaluation on the test set
test_accuracy, test_precision, test_recall, test_f1, test_auc, test_y_scores = evaluate(test_loader)

print(f'Test Acc: {test_accuracy:.2f}% | Test Precision: {test_precision:.4f} | Test Recall: {test_recall:.4f} | Test F1: {test_f1:.4f} | Test AUC: {test_auc:.4f}')

# Plot training curves
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(range(len(train_losses)), train_losses, label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(len(train_accuracies)), train_accuracies, label='Train Accuracy')
plt.plot(range(len(val_accuracies)), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# <center>  Review of the EASTA Model
Based on the ETSA model according to the feedback from our research reviewer, this explanation goes into detail on the requested topics:

## Attention Weights Visualization

1. The ETSA model uses the Edge-Aware and Two-Stage Attention mechanisms to focus on certain nodes and edges during the classification. To visualize these attention weights, we can use the following approach:

2. Extracting Attention Weights: In the first stage, we calculate attention weights by applying a sigmoid function to the hidden node features. In the second stage, we again calculate attention weights by using another sigmoid function over the node features after the first attention mechanism.


3. Displaying Attention Weights: We can visualize the attention weights by highlighting the nodes and edges in a graph. Nodes and edges with higher attention weights can be colored more intensely to indicate their importance in the classification decision. Mathematically,if\ \( \mathbf{h}_i \) represents the hidden features of node \( i \), the attention weight \( \alpha_i \) in the first stage can be computed as:
    \[
    \alpha_i = \sigma(\mathbf{w}_1^T \mathbf{h}_i)
    \]
    where \( \sigma \) is the sigmoid function and \( \mathbf{w}_1 \) is a learnable weight vector.

    Similarly, in the second stage:
    \[
    \beta_i = \sigma(\mathbf{w}_2^T \mathbf{h}_i)
    \]
    where \( \mathbf{w}_2 \) is another learnable weight vector.\\



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torch.optim import Adam
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import to_networkx

# Define the ETSA model with the correct dimensions
class ETSA(nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, num_gcn_layers=1, attention_type='sigmoid', dropout_rate=0.5):
        super(ETSA, self).__init__()
        self.embedding_layer = nn.Linear(num_features, hidden_dim)
        self.num_gcn_layers = num_gcn_layers
        self.gcn_layers = nn.ModuleList([GCNConv(hidden_dim, hidden_dim) for _ in range(self.num_gcn_layers)])

        if attention_type == 'sigmoid':
            self.stage_1_attention = nn.Linear(hidden_dim, 1)
            self.stage_2_attention = nn.Linear(hidden_dim, 1)
        elif attention_type == 'softmax':
            self.stage_1_attention = nn.Linear(hidden_dim, hidden_dim)
            self.stage_2_attention = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(p=dropout_rate)
        self.final_gcn_layer = GCNConv(hidden_dim, num_classes)

    def forward(self, x, edge_index, batch):
        x = self.embedding_layer(x)
        for i in range(self.num_gcn_layers):
            x = F.relu(self.gcn_layers[i](x, edge_index))
            x = self.dropout(x)

        stage_1_attention_weights = torch.sigmoid(self.stage_1_attention(x))
        stage_1_representation = torch.sum(x * stage_1_attention_weights, dim=0)

        stage_2_attention_weights = torch.sigmoid(self.stage_2_attention(x))
        stage_2_representation = torch.sum(x * stage_2_attention_weights, dim=0)

        combined_representation = stage_1_representation + stage_2_representation

        x = self.final_gcn_layer(x, edge_index)

        # Aggregate node embeddings to graph-level embeddings
        x = global_mean_pool(x, batch)

        return x, (stage_1_attention_weights, stage_2_attention_weights)

# Load the MUTAG dataset
dataset = TUDataset(root="./data", name="MUTAG")

# Split the dataset into train, validation, and test sets
torch.manual_seed(12345)
dataset = dataset.shuffle()
num_train = int(0.8 * len(dataset))
num_val = int(0.1 * len(dataset))
num_test = len(dataset) - num_train - num_val

train_dataset = dataset[:num_train]
val_dataset = dataset[num_train:num_train + num_val]
test_dataset = dataset[num_train + num_val:]

# DataLoaders for train, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model, criterion, optimizer
model = ETSA(num_features=dataset.num_node_features, hidden_dim=64, num_classes=dataset.num_classes, num_gcn_layers=2, attention_type='sigmoid', dropout_rate=0.5).to(device)
criterion = torch.nn.CrossEntropyLoss()  # Using CrossEntropyLoss for classification tasks
optimizer = Adam(model.parameters(), lr=0.005)

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    auc = roc_auc_score(y_true, y_pred, average='weighted', multi_class='ovr')
    return accuracy, precision, recall, f1, auc

def fit():
    model.train()
    total_loss = 0
    y_true, y_pred = [], []
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out, _ = model(data.x, data.edge_index, data.batch)

        # Ensure the output and target dimensions match
        out = out.squeeze()  # Remove singleton dimensions
        target = data.y  # Target labels

        assert out.size(0) == target.size(0), f"Output size: {out.size()}, Target size: {target.size()}"

        loss = criterion(out, target)
        total_loss += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()

        y_true.append(target.detach().cpu())
        y_pred.append(out.max(1)[1].detach().cpu())  # Get the index of the max log-probability

    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    accuracy, precision, recall, f1, auc = compute_metrics(y_true, y_pred)
    return total_loss / len(train_loader.dataset), accuracy, precision, recall, f1, auc

@torch.no_grad()
def evaluate(loader):
    model.eval()
    y_true, y_pred = [], []
    for data in loader:
        data = data.to(device)
        out, _ = model(data.x, data.edge_index, data.batch)

        out = out.squeeze()  # Remove singleton dimensions
        target = data.y  # Target labels

        y_true.append(target.cpu())
        y_pred.append(out.max(1)[1].cpu())  # Get the index of the max log-probability

    y_true = torch.cat(y_true, dim=0).detach().numpy()
    y_pred = torch.cat(y_pred, dim=0).detach().numpy()

    accuracy, precision, recall, f1, auc = compute_metrics(y_true, y_pred)
    return accuracy, precision, recall, f1, auc

# Training loop
for epoch in range(301):
    loss, train_accuracy, train_precision, train_recall, train_f1, train_auc = fit()
    val_accuracy, val_precision, val_recall, val_f1, val_auc = evaluate(val_loader)
    if epoch % 50 == 0:
        print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {train_accuracy:.2f}% | Val Acc: {val_accuracy:.2f}% | Val Precision: {val_precision:.4f} | Val Recall: {val_recall:.4f} | Val F1: {val_f1:.4f} | Val AUC: {val_auc:.4f}')

# Final evaluation on the test set
test_accuracy, test_precision, test_recall, test_f1, test_auc = evaluate(test_loader)
print(f'Test Acc: {test_accuracy:.2f}% | Test Precision: {test_precision:.4f} | Test Recall: {test_recall:.4f} | Test F1: {test_f1:.4f} | Test AUC: {test_auc:.4f}')

# Extracting attention weights for visualization
data = next(iter(test_loader)).to(device)
_, (stage_1_attention_weights, stage_2_attention_weights) = model(data.x, data.edge_index, data.batch)

# Visualizing the attention weights using NetworkX
G = to_networkx(data, to_undirected=True)
pos = nx.spring_layout(G)

# Visualizing stage 1 attention weights
node_attention_1 = stage_1_attention_weights.cpu().detach().numpy()
node_sizes_1 = node_attention_1 * 1000  # Scale for better visualization
plt.figure(figsize=(12, 6))
nx.draw(G, pos, with_labels=True, node_size=node_sizes_1, node_color='skyblue', edge_color='gray')
plt.title("Stage 1 Attention Weights")
plt.show()

# Visualizing stage 2 attention weights
node_attention_2 = stage_2_attention_weights.cpu().detach().numpy()
node_sizes_2 = node_attention_2 * 1000  # Scale for better visualization
plt.figure(figsize=(12, 6))
nx.draw(G, pos, with_labels=True, node_size=node_sizes_2, node_color='lightgreen', edge_color='gray')
plt.title("Stage 2 Attention Weights")
plt.show()



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from sklearn.metrics import accuracy_score
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import to_networkx


class ETSA(nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, num_gcn_layers=2, attention_type='sigmoid', dropout_rate=0.5):
        super(ETSA, self).__init__()
        self.embedding_layer = nn.Linear(num_features, hidden_dim)
        self.num_gcn_layers = num_gcn_layers
        self.gcn_layers = nn.ModuleList([GCNConv(hidden_dim, hidden_dim) for _ in range(self.num_gcn_layers)])

        if attention_type == 'sigmoid':
            self.stage_1_attention = nn.Linear(hidden_dim, 1)
            self.stage_2_attention = nn.Linear(hidden_dim, 1)
        elif attention_type == 'softmax':
            self.stage_1_attention = nn.Linear(hidden_dim, hidden_dim)
            self.stage_2_attention = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(p=dropout_rate)
        self.final_gcn_layer = GCNConv(hidden_dim, num_classes)

    def forward(self, x, edge_index):
        x = self.embedding_layer(x)
        for i in range(self.num_gcn_layers):
            x = F.relu(self.gcn_layers[i](x, edge_index))
            x = self.dropout(x)

        stage_1_attention_weights = torch.sigmoid(self.stage_1_attention(x))
        stage_1_representation = torch.sum(x * stage_1_attention_weights, dim=0)

        stage_2_attention_weights = torch.sigmoid(self.stage_2_attention(x))
        stage_2_representation = torch.sum(x * stage_2_attention_weights, dim=0)

        combined_representation = stage_1_representation + stage_2_representation

        x = self.final_gcn_layer(x, edge_index)

        return x, (stage_1_attention_weights, stage_2_attention_weights)

# Load the Cora dataset for two satges visualizations , ask you ask for that we should use the CORA data
cora_dataset = Planetoid(root='data/Cora', name='Cora')

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model, criterion, optimizer
model = ETSA(num_features=cora_dataset.num_node_features, hidden_dim=64, num_classes=cora_dataset.num_classes, num_gcn_layers=2, attention_type='sigmoid', dropout_rate=0.5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)

# Early stopping
best_val_accuracy = 0.0
patience = 10
counter = 0

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Use DataLoader for mini-batch training
    for data in DataLoader(cora_dataset, batch_size=1, shuffle=True):
        data = data.to(device)
        out, _ = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

    # Evaluation on validation set
    model.eval()
    with torch.no_grad():
        val_mask = cora_dataset[0].val_mask
        val_logits, _ = model(cora_dataset[0].x.to(device), cora_dataset[0].edge_index.to(device))
        val_pred_labels = val_logits.argmax(dim=1)[val_mask]
        val_accuracy = accuracy_score(cora_dataset[0].y[val_mask].cpu(), val_pred_labels.cpu())

    # Scheduler step
    scheduler.step(val_accuracy)

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Early stopping check
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered")
            break

# Final evaluation on the test set
model.eval()
with torch.no_grad():
    test_mask = cora_dataset[0].test_mask
    test_logits, (stage_1_attention_weights, stage_2_attention_weights) = model(cora_dataset[0].x.to(device), cora_dataset[0].edge_index.to(device))
    test_pred_labels = test_logits.argmax(dim=1)[test_mask]
    test_accuracy = accuracy_score(cora_dataset[0].y[test_mask].cpu(), test_pred_labels.cpu())
    print(f'Test Accuracy: {test_accuracy:.4f}')

# Visualizing the attention weights using NetworkX
data = cora_dataset[0]
G = to_networkx(data, to_undirected=True)
pos = nx.spring_layout(G)

# Visualizing stage 1 attention weights
node_attention_1 = stage_1_attention_weights.cpu().detach().numpy()
node_sizes_1 = node_attention_1 * 1000  # Scale for better visualization
plt.figure(figsize=(12, 6))
nx.draw(G, pos, with_labels=True, node_size=node_sizes_1, node_color='skyblue', edge_color='gray')
plt.title("Stage 1 Attention Weights")
plt.show()

# Visualizing stage 2 attention weights
node_attention_2 = stage_2_attention_weights.cpu().detach().numpy()
node_sizes_2 = node_attention_2 * 1000  # Scale for better visualization
plt.figure(figsize=(12, 6))
nx.draw(G, pos, with_labels=True, node_size=node_sizes_2, node_color='lightgreen', edge_color='gray')
plt.title("Stage 2 Attention Weights")
plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from sklearn.metrics import accuracy_score
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import to_networkx

# Define the ETSA model
class ETSA(nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, num_gcn_layers=2, attention_type='sigmoid', dropout_rate=0.5):
        super(ETSA, self).__init__()
        self.embedding_layer = nn.Linear(num_features, hidden_dim)
        self.num_gcn_layers = num_gcn_layers
        self.gcn_layers = nn.ModuleList([GCNConv(hidden_dim, hidden_dim) for _ in range(self.num_gcn_layers)])

        if attention_type == 'sigmoid':
            self.stage_1_attention = nn.Linear(hidden_dim, 1)
            self.stage_2_attention = nn.Linear(hidden_dim, 1)
        elif attention_type == 'softmax':
            self.stage_1_attention = nn.Linear(hidden_dim, hidden_dim)
            self.stage_2_attention = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(p=dropout_rate)
        self.final_gcn_layer = GCNConv(hidden_dim, num_classes)

    def forward(self, x, edge_index):
        x = self.embedding_layer(x)
        for i in range(self.num_gcn_layers):
            x = F.relu(self.gcn_layers[i](x, edge_index))
            x = self.dropout(x)

        stage_1_attention_weights = torch.sigmoid(self.stage_1_attention(x))
        stage_1_representation = torch.sum(x * stage_1_attention_weights, dim=0)

        stage_2_attention_weights = torch.sigmoid(self.stage_2_attention(x))
        stage_2_representation = torch.sum(x * stage_2_attention_weights, dim=0)

        combined_representation = stage_1_representation + stage_2_representation

        x = self.final_gcn_layer(x, edge_index)

        return x, (stage_1_attention_weights, stage_2_attention_weights)

# Load the Cora dataset
cora_dataset = Planetoid(root='data/Pubmed', name='Pubmed')

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model, criterion, optimizer
model = ETSA(num_features=cora_dataset.num_node_features, hidden_dim=64, num_classes=cora_dataset.num_classes, num_gcn_layers=2, attention_type='sigmoid', dropout_rate=0.5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)

# Early stopping
best_val_accuracy = 0.0
patience = 10
counter = 0

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Use DataLoader for mini-batch training
    for data in DataLoader(cora_dataset, batch_size=1, shuffle=True):
        data = data.to(device)
        out, _ = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

    # Evaluation on validation set
    model.eval()
    with torch.no_grad():
        val_mask = cora_dataset[0].val_mask
        val_logits, _ = model(cora_dataset[0].x.to(device), cora_dataset[0].edge_index.to(device))
        val_pred_labels = val_logits.argmax(dim=1)[val_mask]
        val_accuracy = accuracy_score(cora_dataset[0].y[val_mask].cpu(), val_pred_labels.cpu())

    # Scheduler step
    scheduler.step(val_accuracy)

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Early stopping check
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered")
            break

# Final evaluation on the test set
model.eval()
with torch.no_grad():
    test_mask = cora_dataset[0].test_mask
    test_logits, (stage_1_attention_weights, stage_2_attention_weights) = model(cora_dataset[0].x.to(device), cora_dataset[0].edge_index.to(device))
    test_pred_labels = test_logits.argmax(dim=1)[test_mask]
    test_accuracy = accuracy_score(cora_dataset[0].y[test_mask].cpu(), test_pred_labels.cpu())
    print(f'Test Accuracy: {test_accuracy:.4f}')

# Visualizing the attention weights using NetworkX
data = cora_dataset[0]
G = to_networkx(data, to_undirected=True)
pos = nx.spring_layout(G)

# Visualizing stage 1 attention weights
node_attention_1 = stage_1_attention_weights.cpu().detach().numpy()
node_sizes_1 = node_attention_1 * 1000  # Scale for better visualization
plt.figure(figsize=(12, 6))
nx.draw(G, pos, with_labels=True, node_size=node_sizes_1, node_color='skyblue', edge_color='gray')
plt.title("Stage 1 Attention Weights")
plt.show()

# Visualizing stage 2 attention weights
node_attention_2 = stage_2_attention_weights.cpu().detach().numpy()
node_sizes_2 = node_attention_2 * 1000  # Scale for better visualization
plt.figure(figsize=(12, 6))
nx.draw(G, pos, with_labels=True, node_size=node_sizes_2, node_color='lightgreen', edge_color='gray')
plt.title("Stage 2 Attention Weights")
plt.show()
